In [2]:
import json
import pennylane as qml
import pennylane.numpy as np

class AbsMagnetization(qml.measurements.StateMeasurement):
    """A measurement class that estimates <|M|>."""

    def process_state(self, state, wire_order):
        """Calculates <|M|>.

        Args:
            state (Sequence[complex]): quantum state with a flat shape. It may also have an
                optional batch dimension.

            wire_order (Wires): wires determining the subspace that the state acts on; a matrix of
                dimension 2**n that acts on a subspace of n wires
        
        Returns: 
            abs_mag (float): <|M|>

        
        See the docs for more information:
        https://docs.pennylane.ai/en/stable/code/api/pennylane.measurements.StateMeasurement.html
        """

        state = qml.state().process_state(state, wire_order)

        # Put your code here #
        n = len(wire_order)
        basis = []
        for i in range(2 ** n):
            sigma_i = [int(b) for b in f"{i:0{n}b}"]
            basis.append(sigma_i)
        M = 0.0
        for i, bstate in enumerate(basis):
            bstate_M = 0.
            for spin in bstate:
                bstate_M += ((abs(state[i]) ** 2) * (1 if spin == 1 else -1))
            M += abs(bstate_M)
        return float(M)


def tfim_ground_state(num_qubits, h):
    """Calculates the ground state of the 1D TFIM Hamiltonian.

    Args:
        num_qubits (int): The number of qubits / spins.
        h (float): The transverse field strength.

    Returns:
        (numpy.tensor): The ground state.
    """


    # Put your code here #
    def reduce(function, iterable, initializer=None):
        it = iter(iterable)
        if initializer is None:
            value = next(it)
        else:
            value = initializer
        for element in it:
            value = function(value, element)
        return value

    def TransverseFieldIsing(N, h):
        id = np.array([[1, 0], [0, 1]])
        sigma_x = np.array([[0, 1], [1, 0]])
        sigma_z = np.array([[1, 0], [0, -1]])

        first_term_ops = np.tile(id, (N, 1, 1))
        first_term_ops[0] = sigma_z
        first_term_ops[1] = sigma_z

        second_term_ops = np.tile(id, (N, 1, 1))
        second_term_ops[0] = sigma_x
        
        H = np.zeros((2**N, 2**N), dtype=float)
        for i in range(N-1):
            H -= reduce(np.kron, first_term_ops)
            first_term_ops = np.roll(first_term_ops, 1, axis=0)
        
        for _ in range(N):
            H -= h * reduce(np.kron, second_term_ops)
            second_term_ops = np.roll(second_term_ops, 1, axis=0)
        return H

    def diagonalize_hamiltonian(H):
        # Diagonalize the Hamiltonian
        eigenvalues, eigenvectors = np.linalg.eigh(H)
        # Sort the eigenvalues and eigenvectors in ascending order
        idx = np.argsort(eigenvalues)
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        return eigenvalues, eigenvectors
    
    H = TransverseFieldIsing(num_qubits, h)
    eigenvalues, eigenvectors = diagonalize_hamiltonian(H)
    i_0 = np.argmin(eigenvalues)
    return eigenvectors[:, i_0]


dev = qml.device("default.qubit")


@qml.qnode(dev)
def magnetization(num_qubits, h):
    """Calculates the absolute value of the magnetization of the 1D TFIM
    Hamiltonian.

    Args:
        num_qubits (int): The number of qubits / spins.
        h (float): The transverse field strength.

    Returns:
        (float): <|M|>.
    """


    # Put your code here #
    state = tfim_ground_state(num_qubits, float(h))
    qml.StatePrep(state, wires=range(num_qubits))
    return AbsMagnetization(wires=list(range(num_qubits)))


def critical_point_estimate(mags, h_values):
    """Provides a finite-size estimate of the critical point of the 1D TFIM
    Hamiltonian. The estimate is done by taking the average value of h for which
    adjacent values of <|M|> differ the most.

    Args:
        mags (numpy.tensor):
            <|M|> values for various values of h (the transverse field strength).
        h_values (numpy.tensor): The transverse field strength values.

    Returns:
        (float): The critical point estimate, h_c.
    """
    differences = [np.abs(mags[i] - mags[i + 1]) for i in range(len(mags) - 1)]
    ind = np.argmax(np.array(differences))

    h_c = np.mean([h_values[ind], h_values[ind + 1]])
    return h_c



In [3]:
tfim_ground_state(2, 0.)

tensor([1., 0., 0., 0.], requires_grad=True)

In [4]:
magnetization(2, 0.)

2.0

In [5]:
h_values = np.arange(0.2, 1.1, 0.3)
[magnetization(5, h) / 5 for h in h_values]

[0.9884299383349955,
 0.9031775982439715,
 0.7311877950729416,
 0.6036208758603537]

In [6]:
num_qubits = 2
h_values = np.arange(0.2, 1.1, 0.005)
mags = []
for h in h_values:
    mags.append(magnetization(num_qubits, h) / num_qubits)
output = critical_point_estimate(np.array(mags), h_values)
print(f"OUTPUT: {output}")

OUTPUT: 0.35250000000000015


In [ ]:
np.vdot([1j, 1j], [1j, 1j])

In [7]:

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    num_qubits = json.loads(test_case_input)
    h_values = np.arange(0.2, 1.1, 0.005)
    mags = []

    for h in h_values:
        mags.append(magnetization(num_qubits, h) / num_qubits)

    output = critical_point_estimate(np.array(mags), h_values)

    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.isclose(solution_output, expected_output, rtol=5e-3)


# These are the public test cases
test_cases = [
    ('5', '0.6735'),
    ('2', '0.3535')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '5'...
Correct!
Running test case 1 with input '2'...
Correct!


In [1]:
import numpy as np

def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value

def TransverseFieldIsing(N, h):
    id = np.array([[1, 0], [0, 1]])
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])
    
    # Vector of operators: [σᶻ, σᶻ, id, ...]
    first_term_ops = np.tile(id, (N, 1, 1))
    first_term_ops[0] = sigma_z
    first_term_ops[1] = sigma_z
    
    # Vector of operators: [σˣ, id, ...]
    second_term_ops = np.tile(id, (N, 1, 1))
    second_term_ops[0] = sigma_x
    
    H = np.zeros((2**N, 2**N), dtype=float)
    for i in range(N-1):
        # Tensor multiply all operators
        H -= reduce(np.kron, first_term_ops)
        # Cyclic shift the operators
        first_term_ops = np.roll(first_term_ops, 1, axis=0)
    
    for i in range(N):
        H -= h * reduce(np.kron, second_term_ops)
        second_term_ops = np.roll(second_term_ops, 1, axis=0)
    
    return H

# Number of spins
N = 8

# Strength of transverse field
h = 1.

# Get the Hamiltonian matrix
H = TransverseFieldIsing(N, h)
print(H)


[[-7. -1. -1. ...  0.  0.  0.]
 [-1. -5.  0. ...  0.  0.  0.]
 [-1.  0. -3. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -3.  0. -1.]
 [ 0.  0.  0. ...  0. -5. -1.]
 [ 0.  0.  0. ... -1. -1. -7.]]


In [ ]:
import numpy as np
from numpy.linalg import eigh

def diagonalize_hamiltonian(H):
    # Diagonalize the Hamiltonian
    eigenvalues, eigenvectors = eigh(H)
    # Sort the eigenvalues and eigenvectors in ascending order
    idx = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    return eigenvalues, eigenvectors

# Number of spins
N = 2

# Strength of transverse field
h = 0.

# Get the Hamiltonian matrix
H = TransverseFieldIsing(N, h)

# Diagonalize the Hamiltonian
eigenvalues, eigenvectors = diagonalize_hamiltonian(H)
i_0 = np.argmin(eigenvalues)

# Print the eigenvalues and eigenvectors
print(f"Ground state energy: {eigenvalues[i_0]}")
print(f"Ground state: {eigenvectors[:, i_0]}")

In [ ]:
np.linalg.eigh